In [2]:
import sys
sys.path.append('/data4/polyakov/instruction_tuning/ToolAlpaca')
from utils import load_openapi_spec, analyze_openapi_spec
import json
from agent.convert_request import call_api_function
from utils import load_openapi_spec, escape
from agent.tools import Tool, GetDetailsTool, tool_projection

def load_tools(api_data):
    # server_url = "http://127.0.0.1:5679"
    server_url = None
    openapi_spec = load_openapi_spec(api_data["Documentation"], replace_refs=True)
    components_descriptions = escape(api_data["Function_Description"]["components"])
    tools = dict()
    for idx, func_name in enumerate(api_data["Function_Projection"]):
        description = escape(api_data["Function_Description"][func_name])
        if idx == len(api_data["Function_Projection"]) - 1:
            description += components_descriptions
        path, method = api_data["Function_Projection"][func_name]
        tools[func_name] = Tool(
            base_url=server_url + "/" + api_data["Name"] if server_url else None,
            func_name=func_name,
            openapi_spec=openapi_spec,
            path=path,
            method=method,
            description=description,
            retrieval_available="retrieval" in api_data.get("external_tools", [])
        )
    return tools

In [3]:
real_tools = json.load(open('/data4/polyakov/instruction_tuning/ToolAlpaca/data/eval_real.json', 'r'))

In [4]:
real_tools[0]

{'Name': 'Nager.Date',
 'Description': 'Public holidays for more than 90 countries',
 'Category': 'Calendar',
 'Authentication': {},
 'external_tools': ['retrieval'],
 'Documentation': '{"openapi": "3.0.1", "info": {"title": "Nager.Date API - V3", "description": "Nager.Date is open source software. If you would like to support the project you can award a GitHub star \\u2b50 or much better <a href=\'https://github.com/sponsors/nager\'>start a sponsorship</a>", "contact": {"name": "Nager.Date on GitHub", "url": "https://github.com/nager/Nager.Date"}, "license": {"name": "MIT License", "url": "https://github.com/nager/Nager.Date/blob/master/LICENSE.md"}, "version": "v3"}, "servers": [{"url": "https://date.nager.at/"}], "paths": {"/api/v3/CountryInfo/{countryCode}": {"get": {"tags": ["Country"], "summary": "Get country info for the given country", "operationId": "CountryCountryInfo", "parameters": [{"name": "countryCode", "description": "Two-character represented country code. For instance

In [5]:
tools = load_tools(real_tools[0])

In [6]:
tools

{'CountryCountryInfo': Tool(name='CountryCountryInfo', description='Get country info for the given country\nParameters: {{"countryCode": "Required. string. Two-character represented country code. For instance, CN or cn represents China."}}\nOutput: Success.\n - Format: text/plain\n - Structure: #CountryInfoDto', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f0f4c22b9d0>, func=<function Tool.__init__.<locals>.func at 0x7f0f4c2ac9d0>, coroutine=None, max_output_len=3000),
 'CountryAvailableCountries': Tool(name='CountryAvailableCountries', description='Get all available countries\nParameters: {{}}\nOutput: Success.\n - Format: text/plain\n - Structure: Array[#CountryV3Dto]', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f0f4c22b9d0>, func=<function Tool.__init__.<locals>.func at 0x7f0f4c2aca60>, coroutine=None, max

In [7]:
test_tool = tools['PublicHolidayIsTodayPublicHoliday']
test_tool._run('{\"countryCode\": \"cn\", \"offset\": 1}')

SSLError: HTTPSConnectionPool(host='date.nager.at', port=443): Max retries exceeded with url: /api/v3/IsTodayPublicHoliday/cn?offset=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1007)')))